# Árboles de Decisión y Derivados

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/oramosul/abd-files/blob/main/spark/3-spark-mllib/4-Clasificacion-Arboles.ipynb)

Se utilizará un conjunto de datos para clasificar algunas universidades como privadas o públicas, con base en los siguientes atributos:
* Apps: Número de aplicaciones (postulaciones) recibidas
* Accept: Número de postulaciones aceptadas
* Enroll: Número de alumnos nuevos inscritos
* Top10perc: Estudiantes nuevos del 10% superior del colegio
* Top25perc: Estudiantes nuevos del 25% superior del colegio
* F.Undergrad: Número de estudiantes de pregrado de tiempo completo
* P.Undergrad: Número de estudiantes a tiempo parcial
* Outstate: Costo de inscripción si no se es del estado
* Room.Board: Costos
* Books: Costos estimados de libros
* Personal: Gasto personal estimado
* PhD: Porcentaje de profesores con Ph.D.
* Terminal: Porcentaje de profesores con grado terminal
* S.F.Ratio: Razón estudiante/profesor
* perc.alumni: Porcentaje de ex-alumnos que realizan donaciones
* Expend: Gasto institucional por estudiante
* Grad.Rate: tasa de graduación

In [ ]:
# Solo si se usa colab
!pip install -q pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [ ]:
# Carga de archivos
!wget -q https://raw.githubusercontent.com/oramosul/abd-files/main/spark/datos/college.csv

In [ ]:
# Cargar los datos
df = spark.read.csv('college.csv', inferSchema=True, header=True)

# Esquema de los datos
# df.printSchema()

In [ ]:
# Algunos valores
df.show(5)

+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+
|              School|Private|Apps|Accept|Enroll|Top10perc|Top25perc|F_Undergrad|P_Undergrad|Outstate|Room_Board|Books|Personal|PhD|Terminal|S_F_Ratio|perc_alumni|Expend|Grad_Rate|
+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+
|Abilene Christian...|    Yes|1660|  1232|   721|       23|       52|       2885|        537|    7440|      3300|  450|    2200| 70|      78|     18.1|         12|  7041|       60|
|  Adelphi University|    Yes|2186|  1924|   512|       16|       29|       2683|       1227|   12280|      6450|  750|    1500| 29|      30|     12.2|         16| 10527|       56|
|      Adrian College|    Yes|1428|  1097|   336|       22|       50|       1036|         99|  

### Pre-procesamiento de Datos

In [ ]:
from pyspark.ml.feature import VectorAssembler

# Ver las columnas disponibles
#df.columns

In [ ]:
# Se tomará todas las columnas, excepto las dos primeras
assembler = VectorAssembler(inputCols=['Apps', 'Accept', 'Enroll', 'Top10perc', 'Top25perc', 'F_Undergrad',
                                       'P_Undergrad', 'Outstate', 'Room_Board', 'Books', 'Personal', 'PhD',
                                       'Terminal', 'S_F_Ratio', 'perc_alumni', 'Expend', 'Grad_Rate'],
                            outputCol="Atributos")

# Transformar los datos
df2 = assembler.transform(df)
df2.show(5)

+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+--------------------+
|              School|Private|Apps|Accept|Enroll|Top10perc|Top25perc|F_Undergrad|P_Undergrad|Outstate|Room_Board|Books|Personal|PhD|Terminal|S_F_Ratio|perc_alumni|Expend|Grad_Rate|           Atributos|
+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+--------------------+
|Abilene Christian...|    Yes|1660|  1232|   721|       23|       52|       2885|        537|    7440|      3300|  450|    2200| 70|      78|     18.1|         12|  7041|       60|[1660.0,1232.0,72...|
|  Adelphi University|    Yes|2186|  1924|   512|       16|       29|       2683|       1227|   12280|      6450|  750|    1500| 29|      30|     12.2|         16| 10527|       56|[2186.0,1924

Creación de la columna que se va a predecir. Se encuentra con valores categóricos, pero se requiere que sea numérica, por lo que se utilizará una indexación.

In [ ]:
# Convertir la columna objetivo (Private: Yes/No) de categórica a indexada
from pyspark.ml.feature import StringIndexer

# Objeto que realiza la indización
indexer = StringIndexer(inputCol="Private", outputCol="Privado")
# Aplicar a los datos
df3 = indexer.fit(df2).transform(df2)

df3.show(5)

+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+--------------------+-------+
|              School|Private|Apps|Accept|Enroll|Top10perc|Top25perc|F_Undergrad|P_Undergrad|Outstate|Room_Board|Books|Personal|PhD|Terminal|S_F_Ratio|perc_alumni|Expend|Grad_Rate|           Atributos|Privado|
+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+--------------------+-------+
|Abilene Christian...|    Yes|1660|  1232|   721|       23|       52|       2885|        537|    7440|      3300|  450|    2200| 70|      78|     18.1|         12|  7041|       60|[1660.0,1232.0,72...|    0.0|
|  Adelphi University|    Yes|2186|  1924|   512|       16|       29|       2683|       1227|   12280|      6450|  750|    1500| 29|      30|     12.2|         

In [ ]:
# Escoger las columnas necesarias para PySpark
df = df3.select("Atributos", 'Privado')

df.show(5)

+--------------------+-------+
|           Atributos|Privado|
+--------------------+-------+
|[1660.0,1232.0,72...|    0.0|
|[2186.0,1924.0,51...|    0.0|
|[1428.0,1097.0,33...|    0.0|
|[417.0,349.0,137....|    0.0|
|[193.0,146.0,55.0...|    0.0|
+--------------------+-------+
only showing top 5 rows



In [ ]:
# Hacer la división en datos de entrenamiento y datos de prueba
df_train, df_test = df.randomSplit([0.7,0.3])

### Clasificadores

In [ ]:
from pyspark.ml.classification import (DecisionTreeClassifier,
                                       GBTClassifier,
                                       RandomForestClassifier)
from pyspark.ml import Pipeline

In [ ]:
# Crear los tres modelos
dt = DecisionTreeClassifier(labelCol='Privado',featuresCol='Atributos', predictionCol='Predicción')
rf = RandomForestClassifier(labelCol='Privado',featuresCol='Atributos', predictionCol='Predicción', numTrees=100)
gb = GBTClassifier(labelCol='Privado',featuresCol='Atributos', predictionCol='Predicción')

In [ ]:
# Entrenar los modelos
modelo_DT = dt.fit(df_train)
modelo_RF = rf.fit(df_train)
modelo_GB = gb.fit(df_train)

### Inspección de Modelos

In [ ]:
preds_DT = modelo_DT.transform(df_test)

#preds_DT.printSchema()
preds_DT.select("Atributos", "Privado", "Predicción").show(5)

+--------------------+-------+----------+
|           Atributos|Privado|Predicción|
+--------------------+-------+----------+
|[141.0,118.0,55.0...|    0.0|       0.0|
|[150.0,130.0,88.0...|    0.0|       0.0|
|[167.0,130.0,46.0...|    0.0|       0.0|
|[191.0,165.0,63.0...|    0.0|       0.0|
|[193.0,146.0,55.0...|    0.0|       0.0|
+--------------------+-------+----------+
only showing top 5 rows



In [ ]:
preds_RF = modelo_RF.transform(df_test)

preds_RF.select("Atributos", "Privado", "Predicción").show(5)

+--------------------+-------+----------+
|           Atributos|Privado|Predicción|
+--------------------+-------+----------+
|[141.0,118.0,55.0...|    0.0|       0.0|
|[150.0,130.0,88.0...|    0.0|       0.0|
|[167.0,130.0,46.0...|    0.0|       0.0|
|[191.0,165.0,63.0...|    0.0|       0.0|
|[193.0,146.0,55.0...|    0.0|       0.0|
+--------------------+-------+----------+
only showing top 5 rows



In [ ]:
preds_GB = modelo_GB.transform(df_test)

preds_GB.select("Atributos", "Privado", "Predicción").show(5)

+--------------------+-------+----------+
|           Atributos|Privado|Predicción|
+--------------------+-------+----------+
|[141.0,118.0,55.0...|    0.0|       0.0|
|[150.0,130.0,88.0...|    0.0|       0.0|
|[167.0,130.0,46.0...|    0.0|       0.0|
|[191.0,165.0,63.0...|    0.0|       0.0|
|[193.0,146.0,55.0...|    0.0|       0.0|
+--------------------+-------+----------+
only showing top 5 rows



### Métricas de Evaluación

In [ ]:
from pyspark.ml.evaluation import (BinaryClassificationEvaluator,
                                   MulticlassClassificationEvaluator)

# Evaluador: usando "exactitud"
evaluadorEX = MulticlassClassificationEvaluator(labelCol="Privado",
                                                predictionCol="Predicción",
                                                metricName='accuracy')

# Evaluador: usando AUC
evaluadorAUC = BinaryClassificationEvaluator(labelCol="Privado",
                                             rawPredictionCol="Predicción",
                                             metricName="areaUnderROC")

In [ ]:
# Mëtricas con árboles de decisión
exactitud_dt = evaluadorEX.evaluate(preds_DT)
auc_dt = evaluadorAUC.evaluate(preds_DT)

print("Usando Árboles de decisión: exactitud={}, AUC={:.3f}".format(exactitud_dt, auc_dt))

Usando Árboles de decisión: exactitud=0.9170305676855895, AUC=0.906


In [ ]:
# Mëtricas con random forest
exactitud_rf = evaluadorEX.evaluate(preds_RF)
auc_rf = evaluadorAUC.evaluate(preds_RF)

print("Usando Random Forest: exactitud={:3f}, AUC={:.3f}".format(exactitud_rf, auc_rf))

Usando Random Forest: exactitud=0.934498, AUC=0.909


In [ ]:
# Mëtricas con gradient boosting
exactitud_gb = evaluadorEX.evaluate(preds_GB)
auc_gb = evaluadorAUC.evaluate(preds_GB)

print("Usando Gradient Boosting: exactitud={:3f}, AUC={:.3f}".format(exactitud_gb, auc_gb))

Usando Gradient Boosting: exactitud=0.912664, AUC=0.885


In [ ]:
modelo_RF.featureImportances

SparseVector(17, {0: 0.0356, 1: 0.0567, 2: 0.1027, 3: 0.0194, 4: 0.0097, 5: 0.253, 6: 0.0804, 7: 0.1848, 8: 0.0592, 9: 0.0061, 10: 0.023, 11: 0.0141, 12: 0.0177, 13: 0.0475, 14: 0.0331, 15: 0.0442, 16: 0.013})

In [ ]:
modelo_GB.featureImportances

SparseVector(17, {0: 0.044, 1: 0.015, 2: 0.02, 3: 0.0347, 4: 0.0255, 5: 0.4436, 6: 0.0189, 7: 0.2297, 8: 0.0305, 9: 0.0074, 10: 0.013, 11: 0.0331, 12: 0.0263, 13: 0.0201, 14: 0.0203, 15: 0.0091, 16: 0.0088})

In [ ]:
modelo_DT.featureImportances

SparseVector(17, {0: 0.024, 4: 0.0316, 5: 0.5579, 6: 0.0184, 7: 0.2874, 8: 0.0394, 10: 0.0087, 11: 0.0125, 14: 0.0201})